# Apriori algorithm

data from provided

In [1]:
import numpy as np
import pandas as pd

# import the data
data = pd.read_csv('dataset.csv', names = ['item_ID', 'time', 'transaction_ID', 'product_ID'], 
                   index_col = 'item_ID')
del data['time']


# get a data of lists of item which each transactions contains
group = data.groupby('transaction_ID')['product_ID'].apply(list)

In [2]:
print(group[:10])

transaction_ID
2             [12, 6, 91, 60, 91]
3        [37, 28, 68, 27, 47, 27]
4                [58, 30, 29, 89]
5                            [94]
6                        [53, 17]
7            [18, 93, 18, 93, 87]
8        [56, 80, 51, 55, 51, 69]
9     [55, 87, 84, 87, 41, 5, 41]
10               [38, 99, 92, 27]
11                   [18, 18, 19]
Name: product_ID, dtype: object


In [3]:
# define a function that used to create a list of single candidate of items
def create_single(group):
    single_list = []
    for transaction in group:
        for item in transaction:
            if not [item] in single_list:
                single_list.append([item])
                
    # make the list a fronzen set easier to handle
    single_list = list(map(frozenset, sorted(single_list)))
    
    return single_list

In [4]:
# define a function that used to get all the support values of the candidate
def scan_transactions(transactions, can_list, min_support):
    can_dict = {}
    for t_id in transactions:
        for can in can_list:
            if can.issubset(t_id):
                if can not in can_dict.keys(): 
                    can_dict[can] = 1
                else: 
                    can_dict[can] += 1
    
    counts = float(len(transactions))
    ret = []
    support_data = {}
    for key in can_dict.keys():
        support_value = can_dict[key] / counts
        if support_value >= min_support:
            ret.insert(0,key)
        support_data[key] = support_value
        
    return ret, support_data

In [5]:
# define a function that used to created a list of multiple candidate itemsets
def combine(target_list, k):
    ret = []
    lenth = len(target_list)
    for i in range(lenth):
        for j in range(i+1, lenth): 
            L1 = sorted(list(target_list[i])[:k-2])
            L2 = sorted(list(target_list[j])[:k-2])
            
            #if first k-2 elements are equal
            if L1==L2: 
                ret.append(target_list[i] | target_list[j])
                
    return ret

In [6]:
# define a function that used to get all the list of candidate item combines and its support value
def calc_support(dataSet, min_support):
    single_list = create_single(dataSet)
    transactions = list(map(set, dataSet))
    ret1, support_data = scan_transactions(transactions, single_list, min_support)
    ret = [ret1]
    k = 2
    while (len(ret[k-2]) > 0):
        retk = combine(ret[k-2], k)
        ret2, sup_k = scan_transactions(transactions, retk, min_support)
        support_data.update(sup_k)
        ret.append(ret2)
        k += 1
        
    return ret, support_data

In [7]:
# define a function that used to calculate the confidence of the support dataset and set a min_conf threshold
def calc_conf(subset, ret, support_data, total_list, min_conf):
    conf_list = []
    for conseq in ret:
        # calculate confidence
        conf = support_data[subset] / support_data[subset-conseq]
        if conf >= min_conf: 
            total_list.append((subset-conseq, conseq, conf))
            conf_list.append(conseq)
            
    return conf_list

In [8]:
# define a function that associate the rules with the original dataset
def calc_rules(subset, ret, support_data, total_list, min_conf):
    m = len(ret[0])
    if (len(subset) > (m + 1)): #try further merging
        mask_ret = combine(ret, m+1)#create Hm+1 new candidates
        mask_ret = calc_conf(subset, mask_ret, support_data, total_list, min_conf)
        if (len(mask_ret) > 1):    #need at least two sets to merge
            calc_rules(subset, mask_ret, support_data, total_list, min_conf)

In [9]:
# define a function used to implement the apriori algorithm
def apriori(combine_list, support_data, min_conf):
    total_list = []
    for i in range(1, len(combine_list)):
        for subset in combine_list[i]:
            ret = [frozenset([item]) for item in subset]
            if (i > 1):
                calc_rules(subset, ret, support_data, total_list, min_conf)
            else:
                calc_conf(subset, ret, support_data, total_list, min_conf)
                
    return total_list 

In [10]:
# since it's a large dataset, the min_support need to be much small
combine_list, support_data = calc_support(group, min_support = 0.0002)

In [11]:
# set the min_conf equal to 0.8, let's say the probability over 80% are always sold together
rules = apriori(combine_list, support_data, min_conf = 0.8)

In [12]:
print('If the target product is sold,',
      'the most possible other product will also be sold is listed in the following\n')
print('Target product_ID', '\tPossible product_ID', '\tProbability')
for i in rules:
    print(i[0],'\t',i[1],'\t',i[2])

If the target product is sold, the most possible other product will also be sold is listed in the following

Target product_ID 	Possible product_ID 	Probability
frozenset({81}) 	 frozenset({32}) 	 0.9295093296475466
frozenset({95}) 	 frozenset({34}) 	 0.8040023543260741
frozenset({53}) 	 frozenset({17}) 	 0.9469259064634789
frozenset({17}) 	 frozenset({53}) 	 0.9504219409282701
frozenset({67, 45}) 	 frozenset({100, 71}) 	 0.8846153846153847
frozenset({67, 29}) 	 frozenset({89, 85}) 	 0.8666666666666666
frozenset({67, 36}) 	 frozenset({85, 94}) 	 0.8799999999999999
frozenset({96, 79}) 	 frozenset({91, 35}) 	 0.8823529411764706
frozenset({16, 18}) 	 frozenset({19, 94}) 	 0.8787878787878789
frozenset({40, 13}) 	 frozenset({96, 91}) 	 0.8378378378378378
frozenset({96, 78}) 	 frozenset({10, 91}) 	 0.8709677419354839
frozenset({36, 76}) 	 frozenset({82, 94}) 	 0.92
frozenset({16, 70}) 	 frozenset({19, 94}) 	 0.8285714285714286
frozenset({63, 39}) 	 frozenset({90, 38}) 	 0.8043478260869564
fr

data from transactions

In [13]:
# import the till system's transactions
tills = pd.read_csv('till_tansactions.csv', header = 0, usecols = [0,3])

#get a data of lists of item which each transactions contains
tills_group = tills.groupby('transationid')['productid'].apply(list)

In [14]:
print(tills_group[:10])

transationid
1         [14, 72, 15]
2      [38, 29, 65, 4]
3         [15, 38, 44]
4     [38, 29, 70, 44]
5           [3, 4, 72]
6          [5, 53, 16]
7         [38, 67, 44]
8         [38, 44, 68]
9         [15, 16, 54]
10                [27]
Name: productid, dtype: object


In [15]:
t_combine_list, t_support_data = calc_support(tills_group, min_support = 0.2)

In [16]:
t_rules = apriori(t_combine_list, t_support_data, min_conf = 0.1)

In [17]:
print('If the target product is sold,',
      'the most possible other product will also be sold is listed in the following\n')
print('Target product_ID', '\tPossible product_ID', '\tProbability')
for i in t_rules:
    print(i[0],'\t',i[1],'\t',i[2])

If the target product is sold, the most possible other product will also be sold is listed in the following

Target product_ID 	Possible product_ID 	Probability
frozenset({38}) 	 frozenset({44}) 	 0.9090909090909092
frozenset({44}) 	 frozenset({38}) 	 1.0
